# Simulation and Analysis Code for $2D$ $O(3)$ Nonlinear Sigma Model with Topological Term $\theta$

## Simulation code for Monte Carlo approach

This approach uses a conventional Monte Carlo simulation with a Metropolis step to simulate the $2D$ $\mathcal{O}(3)$ nonlinear sigma model where the topological term $\theta$ is assigned an imaginary value.

### Code documentation: nonlinearsigma.cpp
** Coming soon **

### Code documentation: lattice.cpp
** last updated November 8, 2023 **

<div class="alert alert-warning">Go through and figure out which - if any - of these functions are never referenced. Move them to a testing class or remove them entirely</div>

#### Constructor <a id='constructor'></a>

```cpp
    Lattice::Lattice(int length, double beta, double itheta){
        Lattice::setLength(length); //set length
        Lattice::setBeta(beta); //set beta
        Lattice::setiTheta(itheta); //set itheta
        Lattice::setnTherm(1000); //set therm steps to default number
        Lattice::setnMC(1000); //set Monte Carlo steps to default number
        Lattice::setFreq(100); //set frequency between saved configs to default number
        Lattice::generateFilename_();
        fixedr_ = false; //this should only be set to true when testing
        use_arcsin_ = true;
    }
```
The constructor takes in the length of the lattice (we are using a square lattice, so length is used for x and y lengths), the value of beta, and the value of itheta. It then sets those three parameters, sets a default number of thermalization steps, Monte Carlo steps, and frequency for saving the lattice configurations and observables.

The functions ```setLength```, ```setBeta```, ```setiTheta```, ```setnTherm```, ```setnMC```, and ```setFreq``` exist to prevent a segmentation fault when running this code. They provide a way into the memory where these parameters are stored, in order to update and save them properly.

<div class="alert alert-warning">Something to consider: should you take in the thermalization steps, MC steps, and freq at this point? What purpose does waiting on those have, or is this a holdover from early testing activities?</div>

The next thing the constructor does is generate the filename, which is described in more detail in [this section](#generateFilename). This way, as soon as the lattice object is constructed, a filename exists for the output.

Then it sets ```fixedr_``` to ```false``` -- this means that we will use the random number generator as usual instead of forcing a pre-set value for the two random numbers (we would only want to fix those numbers in very rare testing conditions). This is currently used in [fixRNG](#fixRNG), [freeRNG](#freeRNG), and [exceptionalConfig](#exceptionalConfig).

<div class="alert alert-warning">What is this doing in exceptionalConfig? Do we need it there?</div>

Finally, it sets our preference for whether to use arccos or arcsin to calculate QL on the triangles -- the default is to use arcsin, but we can change that setting with the function [setTrig](#setTrig)

#### various set param functions <a id = 'setparams'></a>

Each of the functions below allows us to change one of the parameters given for the simulation. These functions are useful when testing the code, as it allows us to update the default values of each of the simulation parameters.

Each of these functions has the same basic steps, which are to update the stored attribute in the object and then generate a new filename (as the parameters are listed in the filename). In addition, ```setLength``` also initializes the lattice, since the length of the lattice determines how many spins are on the lattice and the whole lattice must be reset when the length changes.

```cpp
void Lattice::setLength(int length){
        //tested 6/1/2023
        length_ = length;
        Lattice::generateFilename_();
        Lattice::initialize();
    }
```
```cpp
    void Lattice::setBeta(double beta){
        //tested 6/1/2023
        beta_ = beta;
        Lattice::generateFilename_();
    }
```
```cpp 
    void Lattice::setiTheta(double itheta){
        //tested 6/1/2023
        itheta_ = itheta;
        Lattice::generateFilename_();
    }
```
```cpp   
    void Lattice::setnTherm(int ntherm){
        nTherm_ = ntherm;
        Lattice::generateFilename_();
    }
```
```cpp    
    void Lattice::setnMC(int nMC){
        nMC_ = nMC;
        Lattice::generateFilename_();
    }
```
```cpp    
    void Lattice::setFreq(int freq){
        freq_ = freq;
        Lattice::generateFilename_();
    }
```

#### setTrig <a id = 'setTrig'></a>

This function allows us to change our default use of arcsin to arccos by passing ```use_arcsin = false``` into this function. We use this when performing tests on QL.
```cpp
    void Lattice::setTrig(bool use_arcsin){
        use_arcsin_ = use_arcsin;
    }
```

#### fixRNG <a id = 'fixRNG'></a>

This function allows us to set our random numbers to pre-specified values. This is only used when testing to remove the randomness inherent in the model.
```cpp    
    void Lattice::fixRNG(double r1, double r2){
        //tested 6/1/2023
        fixedr_ = true;
        r1_ = r1;
        r2_ = r2;
    }
```

#### freeRNG <a id = 'freeRNG'></a>

This allows us to undo fixing the random numbers, should we wish to perform a test afterwards that requires a random number.
```cpp   
    void Lattice::freeRNG(){
        //tested 6/1/2023
        fixedr_ = false;
    }
```

#### various get parameter functions <a id = 'getparams'> </a>

The below functions all perform the same operation: fetching the internal attribute.

<div class ="alert alert-warning">Investigate whether this is the proper practice -- I recall it helps prevent seg faults, but also am able to use length_ directly in the for loops later? Is that an internal v external access issue? </div>

```cpp
    int Lattice::getLength(){
        //tested 6/1/2023
        return length_;
    }
    
    double Lattice::getBeta(){
        //tested 6/1/2023
        return beta_;
    }
    
    double Lattice::getiTheta(){
        //tested 6/1/2023
        return itheta_;
    }
    
    int Lattice::getnTherm(){
       return nTherm_; 
    }
    
    int Lattice::getnMC(){
        return nMC_;
    }
    
    std::string Lattice::getFilename(){
        return filename_;
    }
```

#### getFilename <a id = "getFilename"></a>

This fetches the internal class attribute of the filename, which is generated in [generateFilename](#generateFilename). See [this section](#getparams) for more on why we use getFilename instead of referencing the attribute directly.
```cpp
std::string Lattice::getFilename(){
        return filename_;
    }
```
 

#### getPhi <a id = "getPhi"> </a>

This fetches the three-component vector phi at site (i,j).  See [this section](#getparams) for more on why we use getPhi instead of referencing the attribute directly. 
```cpp
   
    Lattice::field Lattice::getPhi(int i, int j){
        //tested 5/30/2023
        return grid_[i][j];
    }
```

#### getRandNums <a id = "getRandNums"> </a>

This fetches the two random numbers most recently used to generate our field vectors. <div class="alert alert-warning">Consider removing this or moving it to a testing class if you make one </div>
```cpp
    double* Lattice::getRandNums(){
        //tested 6/1/2023
        static double r[2] = {r1_, r2_};
        return r;
    }
```

#### getPhiTot <a id = "getPhiTot"> </a>

This calculates the total magnitude of phi on the lattice. 

<div class="alert alert-warning">Currently this is one of our outputs for our observables, but is used nowhere else inside the code. Consider removing this or moving it to a testing class if you make one </div>

```cpp
    double Lattice::getPhiTot(){
        //tested 6/1/2023
        //double phi_tot = 0.;
        double phi_tot(0.);//optimization 7/4/23
        #pragma omp parallel for collapse(2) default(none) shared(length_) reduction(+:phi_tot)
        for(int i = 0; i < length_; i++){
            for(int j = 0; j < length_; j++){
                field phi(Lattice::getPhi(i,j)); 
                phi_tot += dot(phi,phi);
            }
        }
        return phi_tot;
    }
```

#### getAvgG <a id = "getAvgG"> </a>

This fetches the internal attribute which is the average correlation function at site (i,j).

```cpp
    double Lattice::getAvgG(int i, int j){
        //double Gij = Gij_[i][j];
        double Gij(Gij_[i][j]);//optimization 7/4/23
        return Gij;
    }
```

#### clean <a id = "clean"></a>

This function creates a shuffled array of all the lattice sites, so it moves randomly on the lattice.

It sets a limit on number of attempts at fixing exceptional configurations, and then loops over the entire lattice (using the randomized array).

At each site, it checks whether the triangles violate the conditions that allow us to generate integer values for QL. 

If either condition is violated, we mark that configuration as exceptional and generate a new field at the site. 

We then check again, continuing until we find a non-exceptional configuration or we reach our limit.

We then print to the console how many attempts were required to find a non-exceptional configuration. This is just in hopes that we might be able to figure out where things are going particularly poorly.

This function should not be used during the Metropolis step, but only during the initialization step, as the Metropolis step has additional conditions for accepting or rejecting a change to the configuration.


<div class = "alert alert-warning"> To do: make a [removeExceptional](#removeExceptional) function that modularizes that while loop.</div>

```cpp
    void Lattice::clean(){
        //cleaning the lattice means removing exceptional configurations
        int nsites(length_*length_); 
        std::vector<int> site_arr(nsites);
        std::iota(site_arr.begin(), site_arr.end(), 0);     
        shuffle(site_arr.begin(), site_arr.end(), std::default_random_engine(1232));
        
        int exc_lim(1000000);

        for(unsigned int n = 0; n < site_arr.size(); n++){
            int i(site_arr[n]/length_);
            int j(site_arr[n]%length_);
            bool exceptional_config = true;
            int exc_count = 0;
            while (exceptional_config){
                if (Lattice::exceptionalConfig(i,j,0) or Lattice::exceptionalConfig(i,j,1)){
                    exceptional_config = true;
                    exc_count++;
                    //update lattice
                    field phi_new = Lattice::makePhi_();
                    grid_[i][j][0] = phi_new[0];
                    grid_[i][j][1] = phi_new[1];
                    grid_[i][j][2] = phi_new[2]; 
                }
                else{
                    exceptional_config = false;
                }
                if (exc_count > exc_lim){
                    break;
                }
            }//while still exceptional at i,j
            std::cout << "Num attempts at non-exceptional at site (i,j) = ";
            std::cout << i << "," << j << "was " << exc_count << std::endl;
        }//loop over sites
    }
```

#### removeExceptional <a id ="removeExceptional"></a>

#### initialize <a id = "initialize"> </a>

This function first initializes a 2D grid (using C++ vectors), where each entry in our grid is of the custom field type (a 3D array of doubles, representing our 3 components of $\phi$).

It then initializes a similar 2D grid, but each entry is just a double. This will store the value of the correlation function at that point.

Next, it loops over i and j (x and y), and at each site (i,j), it generates a 3-component field phi using the [makePhi](#makePhi) function and saves that field into the 2D grid.

It also initializes the correlation function to 0 at each site (i,j), as the correlation function cannot be calculated yet.

After the loops over i and j are complete, the lattice grid of phi vectors and the lattice grid of correlation function, G, are saved to their respective internal attributes (```grid_``` and ```Gij_```).

Next, the triangles that we use to calculate $Q_L$ are generated, and their coordinates saved. This happens in the [makeTriangles](#makeTriangles) function.

This is the current version of this function:
```cpp
void Lattice::initialize(){
        //tested 5/30/2023
        std::vector < std::vector < Lattice::field > > grid;
        std::vector < std::vector < double > > Gij;
        for(int i = 0; i < length_; i++){
            std::vector <double> Gj;
            std::vector < Lattice::field > gridj;
            for (int j = 0; j<length_; j++){
                field phi = Lattice::makePhi_();
                gridj.push_back(phi);
                Gj.push_back(0.);
            }
            Gij.push_back(Gj);
            grid.push_back(gridj);
        }
        grid_ = grid;
        Gij_ = Gij;
        Lattice::makeTriangles_();
        Lattice::zeroCount();
    }
```

<div class="alert alert-warning"> The function clean() follows this, which goes through the entire lattice and checks for exceptional configurations and tries to remove them. We should consider adding that to initialization itself, and writing a modular function that checks for exceptional configurations that we can use in both the clean() function and in the metropolis step.

Consider also calculating Gij here? You initialize everything to 0, but it's not actually 0. </div>

#### makeTriangles <a id = "makeTriangles"></a>

#### generateFilename <a id='generateFilename'></a>

#### exceptionalConfig<a id = 'exceptionalConfig'></a>

### Code documentation: mathlib.cpp <a id = "mathlib"> </a>
** coming soon **

## Running the simulation code

### Makefile flags

The code is written in C++ and OpenMP and can be compiled with numerous flags.

```bash
USE_OMP ?= TRUE
USE_GPROF ?= FALSE
USE_TEST_PRINT_STATEMENTS ?= FALSE
USE_EXTREME_TEST_CONDITION ?= FALSE
USE_CHECK_QL_COS ?= FALSE
USE_CONST_RN ?= FALSE
```

The first flag "USE_OMP" toggles whether to implement the parallelization in the code. It should be set to "TRUE" if you want to run the simulation in parallel. This is highly recommended for large lattices as the scaling is very poor in series.

If you wish to profile the code, set the second flag "USE_GPROF" to "TRUE". This sets the correct compiler flags so that you can generate the profiling output. To view the output after the code has run, go to the directory in which you have the executable and run the command
```bash
gprof -l nonlinearsigma gmon.out > profiling_results.txt
```
You will then be able to see the profiling report. In general, you should set "USE_GPROF" to false, unless you are looking to optimize the code or troubleshoot it.

The flag "USE_TEST_PRINT_STATEMENTS" activates print statements throughout the code. This is useful for debugging, but should generally be set to FALSE as it slows down the code.

If you run into major problems, set "USE_EXTREME_TEST_CONDITION" to TRUE. This will run a testing suite built into the code, but will not run the usual simulation. This can help you identify problems in the code, and the testing suite is a function inside the main function, which can be modified as needed to add more tests.

To switch from using arcsin to calculate $Q_{L}$ to using arccos, set "USE_CHECK_QL_COS" to TRUE. In general, this should be set to FALSE< as we want to use arcsin due to its useful symmetry.

Finally, if you want to remove the random number generation and use a constant value for the random numbers, set "USE_CONST_RN" to true. This should only be done when testing the code.

### Compiling the code

Once you have set the flags you want to use, you can compile the code by typing

```bash
make -f make_sigma
```
It can be useful to run 

```bash
make -f make_sigma clean
```
first to clear out old .o files that might not be updated otherwise.

Once you have compiled the code, you should have an executable by the name of ```nonlinearsigma``` which you will use to run the simulation.

### Other important things to note before running the code

The simulation requires a list of parameters. We give those to the code through the text file ```inputs.txt```. That file consists of a list of parameter keywords and a value. Be very careful not to change the format of this document, only the numbers, otherwise the simulation won't be able to understand what values get assigned to what parameters. 

The file ```inputs.txt``` should look like this

```
L = 10
beta = 1.6
itheta = 1.
ntherm = 1000
nMC = 1000
freq = 100
```
Where ```L``` gives the length of the square lattice, ```beta``` is $\beta = 1/g_{L}$ and should be set to 1.6, ```itheta``` is the imaginary value given for the topological term and is given in fractions of $\pi$ (so you should enter 0.5 if you want $i \theta = \pi/2$), ```ntherm``` is the number of steps you want the simulation to take for thermalization, ```nMC``` is the number of steps in the Monte Carlo loop after thermalization, and ```freq``` sets the number of steps between saved configurations.

### Running the code -- single job in interactive node

If you want to run this in an interactive node to test, you can request an interactive node with the following command:

```bash
salloc --cpus-per-task=1 --time=00:30:00
```
where you can adjust the number of cpus per task if you want to run the parallelized code and the time requested is in format hh:mm:ss

You can run the job using the command

```bash
./nonlinearsigma inputs.txt
```
```nonlinearsigma``` is the name of the executable created when you compiled the code, and inputs.txt is the list of inputs mentioned above. 

### Running the code -- single SLURM submission 

Once you're comfortable with the code and want to submit a job that's longer than your interactive session (or one that may need to run overnight, etc), you do that by writing a SLURM script and sending that script to the scheduler.

The script is called ```submit_sigma.sh``` and looks like this:
```bash
#SBATCH --job-name=nonlinearsigma_omp_test           # Job name
#SBATCH --mail-type=ALL                              # Mail events (NONE, BEGIN, END, FAIL, ALL)
#SBATCH --mail-user=cberger@smith.edu                # Where to send mail
#SBATCH --partition=phyq                             # Which partition to use
#SBATCH --nodes=1                                    # Number of nodes
#SBATCH --cpus-per-task=1                           # Number of threads per task (OpenMP)
#SBATCH --mem=1gb                                    # Job memory request
##SBATCH --time=05:00:00                             # Time limit hrs:min:sec
#SBATCH --output=nonlinearsigma_omp_test_%j.log      # Standard output 
#SBATCH --error=err_nonlinearsigma_omp_test_%j.log   # Standard output and error log

pwd; hostname; date

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

echo "Running nonlinear sigma on single CPU core"

/usr/bin/time -v ./nonlinearsigma inputs.txt

date
```

Let's walk through what this script does.

The first line:
```bash
#SBATCH --job-name=nonlinearsigma_omp_test           # Job name
```
Assigns a name to the job, which can help you keep track of what is running in the queue. I tend to name all my jobs in a certain phase of the work the same thing (e.g. "nonlinearsigma_small_L_tests" if I'm testing out the script on small lattices or "nonlinearsigma_first_production_run" if I'm starting to take data for real). This is just for your own information, so name it whatever you'd like.

The next two lines
```bash
#SBATCH --mail-type=ALL                              # Mail events (NONE, BEGIN, END, FAIL, ALL)
#SBATCH --mail-user=cberger@smith.edu                # Where to send mail
```
tell SLURM who to email and when to email you. I have it set to email me at my Smith email anytime a job starts, ends, or fails. I find this helpful, especially if trouble arises, but it's your choice what to put here (just don't leave my email address in!).

The next line
```bash
#SBATCH --partition=phyq                             # Which partition to use
```
sends the jobs to the partition that belongs to the physics department. We have priority on this node, but we can request time on other nodes if we really need to. We'd need to talk to CATS about that if we wanted to do it.

The next lines specify the code's needs:
```bash
#SBATCH --nodes=1                                    # Number of nodes
#SBATCH --cpus-per-task=1                           # Number of threads per task (OpenMP)
#SBATCH --mem=1gb                                    # Job memory request
```
Since we are using OpenMP for parallelization, we only need one node, but we will want to change ```bash --cpus-per-task``` to something larger for OpenMP. Unless doing a very large lattice (L > 100), I tend to ask for 30 CPUs per task, which allows for 2 jobs to run at a time on each node. If you're doing something large, you might want to consider asking for 60 CPUs per task, which will occupy an entire node.

This line
```bash
##SBATCH --time=05:00:00                             # Time limit hrs:min:sec
```
sets a time limit -- it will cut off your code when that limit is reached, whether it is done or not. Note there is an extra `#` here -- that means it's commented out, so it will not have a time limit. I tend to only use the time limits when testing.

These next two lines
```bash
#SBATCH --output=nonlinearsigma_omp_test_%j.log      # Standard output 
#SBATCH --error=err_nonlinearsigma_omp_test_%j.log   # Standard output and error log
``` 
give names to the output and error files. These files are created when the code runs and can help you debug if things go wrong.

All the above were instructions for SLURM, which schedules jobs on the machine's available resources. Now we get into the commands to actually run the code.

```bash
pwd; hostname; date
```
This just prints where the job is being run from and the date and time before the job starts.

```bash
export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK
```
this sets the number of threads in OpenMP equal to the number of CPUs per task you assigned above.

```bash
echo "Running nonlinear sigma"

/usr/bin/time -v ./nonlinearsigma inputs.txt
```
This prints out an annoucement that you're starting the job, which is useful in the logfile, and then runs the simulation with the inputs file. Make sure you have the inputs file and the executable in the same folder as the SLURM script so the computer can find them.

And finally
```bash
date
```
we print the datetime stamp at the end of the simulation.

### Running the code -- batch SLURM submissions

Each inputs file is one set of parameters, and we need to get lots of data. If you're not interested in manually setting up these SLURM scripts, input files, etc by hand, I don't blame you. That's why I wrote a Python code to do most of the work for you.

In this code, you specify what parameters you want to run, and the script creates all the appropriate directories and puts the correct inputs file, slurm script, and the executable in each directory. You still have to go in and submit the files yourself, but it's much easier than writing all these scripts yourself.

The script is called ```create_input_files.py``` and here is the part you will need to modify:

```python
#beta = 1/g = 1.6
beta = 1.6
#number of steps in thermalization
ntherm = 4000
#number of monte carlo steps
nMC = 10000
#number of steps between samples
freq = 100
#list of values for lattice length L
L_list = [10,40,80,120,180]
#list of values for itheta (as fractions of pi)
itheta_list = [0.0,0.125,0.25,0.375,0.5,0.625,0.75,0.875,1.,1.125]

script_name = "nonlinearsigma"
job_name = "nlsigma_prelim_tests"
email = "cberger@smith.edu"
num_cpus = 30

```
Beta, ntherm, nMC, and freq all take one number as input, but you can create a list of the number of lattice lengths you want, and the values for itheta (remember these are fractions of pi). 

```script_name``` is the executable, so it should be "nonlinearsigma", but ```job_name``` is what will be put in for the job name in the SLURM script. Similarly, you can enter the email address you want included in the SLURM script and choose how many CPUs you want.

Once you've modified this script to have the values you want, put it in a directory for this batch. I tend to name those something like ```run_yyymmdd```. Also in that directory should be the executable, so copy that in once you've compiled the code.

Then, inside the batch directory, go ahead and run the python script

```bash
python create_input_files.py
```

When it's done, you should see the subdirectories created -- one for each job. You need to go into each subdirectory to submit the jobs using

```bash
sbatch submit_sigma.sh
```
and it will submit the jobs to the scheduler.

You can run this with 
```bash
sbatch submit_sigma.sh
```
and you can check the status of your jobs any time with the command
```bash
squeue -u your_username
```

## Python Code for Analysis

There are a number of Jupyter Notebooks prepared to explore the data that comes out of the simulation. These are all located in the Analysis folder. There is also a Python class, LatticeData.py, which creates an analyzer object that has internal functions for analysis and visualization.

### LatticeData.py - Data Analyzer Class

#### Importing and initializing

This is a Python class. You can create a lattice data object, which then holds all the functionality you need to analyze and visualize the data. You will need to import this into your notebook like this:
```python
from LatticeData import *
```

You will want to also import the following:
* Numpy
* Pandas
* Matplotlib
* Seaborn

The simplest way to start now is by initializing the object:

``` python
analyzer = LatticeData()
```
You can now do a number of operations with this analyzer. The full list of functions is included later, but if you wanted to get all the data from that folder and put it into a Pandas dataframe, that would look like this:

```python
df = analyzer.get_data()
```

There are a number of default settings in this class, which you can change when you initialize. The defaults are:
```python
analyzer_default = LatticeData(datadir = "/data/", header = "nonlinearsigma_data",
                 dirheader = "nlsigma_data", Gheader = "Gij_avg_nonlinearsigma_data", 
                 tol = 0.00001, palette = "viridis")
```
which initializes the following internal variables:
```python

self.path = os.getcwd()+datadir #location of data
self.header = header #set the start of the filename for the data files
self.dirheader = dirheader #set the start of the data directory name from the runs
self.Gheader = Gheader #set the start of the filename for correlation function files
self.tol = tol #set the error range for parameters -- this is for filtering
self.palette = palette #option to change seaborn palette
self.observables = ['Q_L', 'A_L', 'S_L', 'Xi_L'] #observables whose expectation values can be computed
self.parameters = ["itheta", "beta", "length","nMC", "ntherm", "freq"] #parameters read in by the simulation code
```
Most of these will not need to be changed, but let's say you want to analyze a special batch of data, which you've stored in a directory called ```data_test```, and you want to use a different visualization palette, you could intialize the object like this:
```python
analyzer_special = LatticeData(datadir = "/data_test/", palette = "magma")
```
(You can choose any [seaborn palette](https://seaborn.pydata.org/tutorial/color_palettes.html) for this)

Then when you run the function
```python  
df = analyzer_special.get_data()
```
it will aggregate all the data files in the folder ```data_test```.

Below is a complete list of functions for the LatticeData class, with a brief description. While Python doesn't have the same public/private distinctions as C++, I've organized them into those same groups. Public functions are things that you may want to use. Private functions are functions that you should never need to call yourself, but are called internally.

#### Lattice Data Class Built-In Functions ("Public" or external)

##### copy_data_from_directory

```python
copy_data_from_directory(self, src_dir, dst_path = None)
```
This function loops through a specified directory, finds any simulation directories (directions that begin with "nlsigma_data" or whatever you have specified under ```dirheader``` in your intialization), and copies the .csv files within those directories to some destination directory. The default destination directory is whatever you've specified for your data directory.
    The function requires a source directory to be passed as a string -- this is the directory where you have all your simulation results that you want copied over -- and gives you the option to specify a different destination directory using ```dst_path```)
    
If some of your simulations are not complete yet (determined by testing whether the .csv has the correct number of lines), this function will not copy those files and will print out the name of the run and how many lines there are in the data output file. For example, the folder ```run_7_18_23_stats``` contains some runs that haven't finished yet. If I try to copy the data from that directory into my data directory:
    
```python
analyzer.copy_data_from_directory("run_7_18_23_stats")
```
here's what appears printed out:
```
run L_180_beta_1.600000_itheta_0.000000_ntherm_5000_nMC_50000_freq_100 not yet complete: 378 lines
run L_180_beta_1.600000_itheta_2.356194_ntherm_5000_nMC_50000_freq_100 not yet complete: 391 lines
run L_180_beta_1.600000_itheta_3.141593_ntherm_5000_nMC_50000_freq_100 not yet complete: 237 lines
run L_180_beta_1.600000_itheta_1.570796_ntherm_5000_nMC_50000_freq_100 not yet complete: 381 lines
run L_180_beta_1.600000_itheta_0.785398_ntherm_5000_nMC_50000_freq_100 not yet complete: 240 lines
```
And those runs will not be in the data directory, while completed runs will have been copied in.

##### all_params

```python
all_params(self)
```
This function collects every unique set of parameters from your default data directory and returns it as a dataframe. It does not take any inputs -- if you want to know what combinations of parameters are in your directory, this function will tell you.
    
```python
params = analyzer.all_params()
params.head()
```
```
>
    freq	nMC	ntherm	itheta	beta	length
0	100.0	50000.0	5000.0	0.785398	1.6	20.0
1	100.0	50000.0	5000.0	0.000000	1.6	80.0
2	100.0	50000.0	5000.0	0.000000	1.6	20.0
3	100.0	50000.0	5000.0	1.570796	1.6	40.0
4	100.0	50000.0	5000.0	2.356194	1.6	10.0
```
It also adds a column that tells you what fraction of $\pi$ $i\theta$ is, to make it easier to determine what new jobs should be run

If you want to collect data from more than one run, you can do this by only specifying which parameters you want in your dataframe, and the function will filter the data accordingly.

```python
param_dict = {"length": 10, "itheta":2.356194}
filtered_data = analyzer.get_data(single_run = False, suppress_output = True, **param_dict)
```

If you want *all* the data, just leave out the parameter dictionary entirely and it won't filter anything.

```python
all_data = analyzer.get_data(single_run = False, suppress_output = True)
```

NOTE: I strongly recommend suppressing output if you are collecting more than 2 or 3 runs, as it will slow the program down and produce a flood of output.

##### get_data

```python
get_data(self, single_run = False, corr = False, suppress_output = True, **kwargs)
```

This function will collect raw data from one or more runs and return it as a Pandas dataframe. 

If you want to just get data from one simulation run (e.g. to check thermalization or autocorrelation), you should set ```single_run``` to ``` True```. If you want the correlation function data from that run, you should set ```corr``` to ```True``` -- otherwise it will return the observable data. If you want it to print out the parameter sets it's putting into the dataset, set ```suppress_output``` to ```False```

You then need to specify what the parameters are for the run you want to see. You do this by creating a dictionary. When selecting a single one, you must ensure your dictionary has *all* the parameter values specified. The keys for these values are:
* "length": length of the lattice in each direction
* "itheta": value of the imaginary value used for theta -- actual number here, not an integer multiple of pi, but you can always use ```np.pi``` to specify it
* "beta": for our purposes this will always be 1.6, but you need to specify it anyway
* "nMC": number of steps in the Monte Carlo loop 
* "ntherm": number of steps in the thermalization loop
* "freq": frequency with which the configurations were saved

If you forget one of these, the function will remind you: 

```python
param_dict = {"length": 10, "freq": 100, "itheta":2.356194, "beta":1.6, "nMC":50000}
one_run = analyzer.get_data(single_run = True, suppress_output = False, **param_dict)

```
```
Missing parameters in input: 
['ntherm']
```
So you know now to add in the "ntherm" you're looking for. Now it should work:
```python
param_dict = {"length": 10, "freq": 100, "itheta":2.356194, "beta":1.6, "nMC":50000, "ntherm": 5000}
one_run = analyzer.get_data(single_run = True, suppress_output = False, **param_dict)
```
```
freq 100
nMC 50000
ntherm 5000
itheta 2.356194
beta 1.6
length 10
```

```python
one_run.head(3)
```
```
	step	|phi|	Q_L	A_L	S_L	Xi_L	F_LRe	F_LIm	acc	dt	...	Q_L_ta	A_L_ta	S_L_ta	Xi_L_ta	corr_length_Re	corr_length_Im	F_Re_py	F_Im_py	mass_gap_Re	mass_gap_Im
0	0	100.0	0.479179	-183.166830	-184.295868	45.724903	3.675149	8.145972	0.240000	0.0	...	1	3	3	4	8.966957	-3.640801	0.916337	0.890996	0.095738	0.038872
1	100	100.0	0.159302	-195.846927	-196.222275	62.369185	3.675149	8.145972	0.203762	0.0	...	1	3	3	4	10.472580	-4.252121	0.916337	0.890996	0.081974	0.033283
2	200	100.0	0.000000	-199.012224	-199.012224	48.119359	3.675149	8.145972	0.207164	0.0	...	1	3	3	4	9.198745	-3.734913	0.916337	0.890996	0.093325	0.037892
3	300	100.0	0.312655	-179.261053	-179.997729	46.124447	3.675149	8.145972	0.205681	0.0	...	1	3	3	4	9.006048	-3.656673	0.916337	0.890996	0.095322	0.038703
```

##### do_stats

```python 
do_stats(self, therm = 0., stack = False, **kwargs)
```
This function collects data from the directory you specified when you initialized the object. If you want to filter the data by parameters, you just need to enter a parameter dictionary just like when using get_data() above, and it will filter to include only data that matches the parameter values you've entered.

Once all the raw data is collected, it will perform some basic statistical analysis. All observables will have a mean and standard error calculated from data after thermalization. If you need to change the thermalization point, you can increase it by changing the ```therm``` argument in the function -- give it the fractional value of the data you want it to drop from the beginning of the dataset. So for example, if you have a run with ```nMC = 1000``` and ```freq = 10```, you will have a total of 100 steps in your dataset. If you set ```therm = 0.2```, it will drop the first 20 steps before computing the mean and standard error.

After calculating means and standard errors, it also computes the autocorrelation time (the step at which the observable's autocorrelation value drops below 0.3), and then determines how long that run took to complete, saving that information in seconds, minutes, and hours.

It saves all of this along with the parameters used in that run. The result is a very comprehensive dataframe. Here's an example of it run on a data directory with 25 runs in it:

```python
df_stats = analyzer.do_stats()
df_stats.info()
```
```
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   length               25 non-null     float64
 1   itheta               25 non-null     float64
 2   beta                 25 non-null     float64
 3   nMC                  25 non-null     float64
 4   ntherm               25 non-null     float64
 5   freq                 25 non-null     float64
 6   |phi|_mean           25 non-null     float64
 7   Q_L_mean             25 non-null     float64
 8   A_L_mean             25 non-null     float64
 9   S_L_mean             25 non-null     float64
 10  Xi_L_mean            25 non-null     float64
 11  F_LRe_mean           25 non-null     float64
 12  F_LIm_mean           25 non-null     float64
 13  acc_mean             25 non-null     float64
 14  Q_L_ta               25 non-null     float64
 15  A_L_ta               25 non-null     float64
 16  S_L_ta               25 non-null     float64
 17  Xi_L_ta              25 non-null     float64
 18  corr_length_Re_mean  25 non-null     float64
 19  corr_length_Im_mean  25 non-null     float64
 20  F_Re_py_mean         25 non-null     float64
 21  F_Im_py_mean         25 non-null     float64
 22  mass_gap_Re_mean     25 non-null     float64
 23  mass_gap_Im_mean     25 non-null     float64
 24  |phi|_std            25 non-null     float64
 25  Q_L_std              25 non-null     float64
 26  A_L_std              25 non-null     float64
 27  S_L_std              25 non-null     float64
 28  Xi_L_std             25 non-null     float64
 29  F_LRe_std            25 non-null     float64
 30  F_LIm_std            25 non-null     float64
 31  acc_std              25 non-null     float64
 32  corr_length_Re_std   25 non-null     float64
 33  corr_length_Im_std   25 non-null     float64
 34  F_Re_py_std          25 non-null     float64
 35  F_Im_py_std          25 non-null     float64
 36  mass_gap_Re_std      25 non-null     float64
 37  mass_gap_Im_std      25 non-null     float64
 38  time (sec)           25 non-null     float64
 39  time (min)           25 non-null     float64
 40  time (hr)            25 non-null     float64
dtypes: float64(41)
memory usage: 8.1 KB

```

If I only want the analyzed data for runs where the lattice had a length of 20, I could modify this as follows:

```python
df_stats = analyzer.do_stats(**{"length":20})
df_stats.info()
```
```
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   length               5 non-null      float64
 1   itheta               5 non-null      float64
 2   beta                 5 non-null      float64
 3   nMC                  5 non-null      float64
 4   ntherm               5 non-null      float64
 5   freq                 5 non-null      float64
 6   |phi|_mean           5 non-null      float64
 7   Q_L_mean             5 non-null      float64
 8   A_L_mean             5 non-null      float64
 9   S_L_mean             5 non-null      float64
 10  Xi_L_mean            5 non-null      float64
 11  F_LRe_mean           5 non-null      float64
 12  F_LIm_mean           5 non-null      float64
 13  acc_mean             5 non-null      float64
 14  Q_L_ta               5 non-null      float64
 15  A_L_ta               5 non-null      float64
 16  S_L_ta               5 non-null      float64
 17  Xi_L_ta              5 non-null      float64
 18  corr_length_Re_mean  5 non-null      float64
 19  corr_length_Im_mean  5 non-null      float64
 20  F_Re_py_mean         5 non-null      float64
 21  F_Im_py_mean         5 non-null      float64
 22  mass_gap_Re_mean     5 non-null      float64
 23  mass_gap_Im_mean     5 non-null      float64
 24  |phi|_std            5 non-null      float64
 25  Q_L_std              5 non-null      float64
 26  A_L_std              5 non-null      float64
 27  S_L_std              5 non-null      float64
 28  Xi_L_std             5 non-null      float64
 29  F_LRe_std            5 non-null      float64
 30  F_LIm_std            5 non-null      float64
 31  acc_std              5 non-null      float64
 32  corr_length_Re_std   5 non-null      float64
 33  corr_length_Im_std   5 non-null      float64
 34  F_Re_py_std          5 non-null      float64
 35  F_Im_py_std          5 non-null      float64
 36  mass_gap_Re_std      5 non-null      float64
 37  mass_gap_Im_std      5 non-null      float64
 38  time (sec)           5 non-null      float64
 39  time (min)           5 non-null      float64
 40  time (hr)            5 non-null      float64
dtypes: float64(41)
memory usage: 1.7 KB
```
Notice we now only have 5 entries in our dataframe, not 25. We can check that this worked:

```python
df_stats["length"].unique()
```
```
array([20.])
```

If you want the data returned using Pandas MultiIndex, set ```stack``` to ```True```, but MultiIndex doesn't always play well with seaborn and other plotting tools, so the default is ```False```.

##### get_plot_data

```python
get_plot_data(self, obs = "Q_L", L = 10, beta = 1.6, nMC = 10000, ntherm = 1000, freq = 100, stack = False)
```
This function allows you to get the analyzed data for one observable (you may specify which one, but the default is ```Q_L```) as a function of itheta in order to plot it. You must specify a single value for all other parameters (```L```, ```beta```, ```nMC```, ```ntherm```, ```freq```) or leave them blank to use the defaults.

This pulls its data from the internally stored dataframe self.df_stats, which is created when you run the do_stats function. If you have not run that function yet, it will do it for you, with the default settings of ```therm = 0.0``` and ```stack = False```.

<div class="alert alert-warning">
This doesn't play nice with MultiIndex right now, so I recommend making sure to do this with the default of 
   
```python
stack = False
```
</div> 

This function could be modified in order to choose your independent variable, but right now all the plots we are interested in are functions of itheta, so it's unneccesary to plot the observable as a function of any other parameter. Seaborn can be used with the raw data to study systematic effects or other things that may be functions of ```nMC```, ```ntherm``` or ```L```. 

This returns three items: x, y, and y_err, which can then be plotted immediately with the matplotlib errorbar function.

For example, if you wanted to plot ```Q_L``` as a function of ```itheta``` for all the different lengths you have in your data, you would do the following:

```python
params = analyzer.all_params()
lengths = params["length"].unique()
colors = sns.color_palette("Blues", len(lengths))
observable = "Q_L"

for n,length in enumerate(lengths):
    x,y,err = analyzer.get_plot_data(obs = observable, L = length, beta = 1.6, nMC = 50000, 
                                     ntherm = 5000, freq = 100)
    plt.errorbar(x, y , yerr = err, marker = ".", ls = "none", color = colors[n], label ="L="+str(length))
plt.legend()
plt.title(observable)
plt.show()
```
And you would get the following output:

<img src = "./Figs_README/QL_v_itheta_example.jpg">

And if you wanted to plot the magnetic susceptibility ```Xi_L``` as a function of ```itheta``` for each length, you would do:

```python
params = analyzer.all_params()
lengths = params["length"].unique()
colors = sns.color_palette("Reds", len(lengths))
observable = "Xi_L"

for n,length in enumerate(lengths):
    x,y,err = analyzer.get_plot_data(obs = observable, L = length, beta = 1.6, nMC = 50000, 
                                     ntherm = 5000, freq = 100)
    plt.errorbar(x, y , yerr = err, marker = ".", ls = "none", color = colors[n], label ="L="+str(length))
plt.legend()
plt.title(observable)
plt.show()
```
And you would get the following output:

<img src = "./Figs_README/XiL_v_itheta_example.jpg">

##### get_corr_func

```python
get_corr_func(self,suppress_output = False,**kwargs)
```

This function returns the average correlation function for the set of parameters specified. This requires returning one single run, so it plays by the same rules as ```get_data``` with ```single_run = True``` and it will tell you if you left out a parameter:

```python
itheta = np.pi
beta = 1.6
length = 20
nMC = 50000
corr_params = {"itheta": itheta, "beta": beta,"length": length,"nMC": nMC}
G_ij = analyzer.get_corr_func(suppress_output = False, **corr_params)
```

```
Missing parameters in input: 
['ntherm', 'freq']

```
When you specify the complete set of parameters, it returns a 2D numpy array that represents the average correlation function on each lattice site. You can then plot this with imshow:
```python
plt.imshow(G_ij, cmap = "viridis", aspect='equal')
plt.colorbar()
plt.title("Correlation function for L = "+str(corr_params["length"])+", itheta = "+str(corr_params["itheta"]))
plt.show()

```
and you will get something that looks like this:


<img src = "./Figs_README/Gij_example.jpg">

<div class="alert alert-warning">
Currently, the simulation returns the average value of the correlation function computed after thermalization. There is no way to change this after the simulation is run, and we don't have the code set up yet to return any error on the correlation function. That may come in future versions of the code.
</div> 

#### Lattice Data Class Built-In Functions ("Private" or internal)

##### get_data_files

```python
get_data_files(self, corr = False)
```
This function is referenced by ```all_params``` and ```get_data``` (which are both in turn called by other functions). 

It loops through every file in the data directory (```self.path```) set when you initialize the object. If you have set ```corr``` to ```True``` (e.g. as done in the function ```get_corr_func```), it uses ```self.Gheader``` to pick out all the simulation files that are correlation function data and put them in a list. If ```corr``` is ```False```, it uses ```self.header``` to pick out all the regular data files from the simulation and create a list of those.

It returns just a list of all the relevant data filenames, which can then be used by other functions to extract the parameters (as in ```all_params```) or make a dataframe (as in ```get_data```).

##### get_file_params

```python
get_file_params(self, file)
```
This function is referenced by ```all_params``` and ```get_data``` (which are both in turn called by other functions). 

It takes a filename (e.g. one from the list generated by ```get_data_files```) and extracts from that filename the parameter values used in the simulation. It returns a dictionary with the following keys:

* "freq" -- the number of steps between saved configurations
* "nMC" -- the number of steps in the Monte Carlo loop
* "ntherm" -- the number of thermalization steps before beginning the Monte Carlo loop
* "itheta" -- the value of the parameter $i \theta$ (actual value, not integer multiples of pi)
* "beta" -- inverse lattice coupling (currently set to 1.6 for all simulations, but could be changed)
* "length" -- number of lattice sites in both x and y


##### in_list

``` python
in_list(self,pdict,**kwargs)
```
This function is only called by ```get_data``` but it serves the important function of managing all the filtering we want to it. It checks each parameter fed into ```get_data``` in the keyword arguments and it checks it against the parameters in the filename for a simulation run. If all the parameter values specified in the keyword arguments match parameters in the file, it returns ```True```, otherwise it returns ```False```. 

##### calc_F

```python
calc_F(self, **kwargs)
```

This function calculates the correlation function at the smallest nonzero lattice momentum ($p_{0} = 2 \pi/L$) . It requires the correlation function for that set of parameters ($G$, which it obtains by calling ```get_corr_func```) and then performs the following calculation, summing over all lattice sites $(x,y)$:

$\mathcal{F} = \frac{1}{2}\sum_{x,y}(e^{2 \pi i x/L} + e^{2 \pi i y/L})G(x,y)$

##### calc_corr_length

```python
calc_corr_length(self,Xi,L,F_py)
```

### Analysis Notebooks

#### AnalysisTesting.ipynb

#### PhiDist.ipynb

#### DataComparison.ipynb

#### SystematicsAndTiming.ipynb

#### CorrelationFunction.ipynb

#### Observables.ipynb

## Summary of Preliminary Results

Preliminary simulation code is being run on small lattices ($L = 10,40,80$) with the intent to scale to larger lattices in the next few months.

Early results look promising. For example, when comparing our results for the topological charge QL to those of Allés et al, we see a similar trend in the data:

<img src = "./Figs_README/AllesPRBQ_L.png" width="425"/><img src = "./Figs_README/Q_L_preliminary.jpg" width="425"/>

We expect as the size of the lattice grows (to 180 x 180, as was used in the PRB paper), our $Q_L$ plot should match theirs. Simulations at L = 180 are currently underway.

## Current issues and open questions

Issues we are working on resolving:
* The topological charge calculation ($Q_{L}$) is currently returning non-integer values
* The correlation function at the smallest nonzero lattice momentum $2\pi/L$ is a complex number, therefore so is our correlation length. Correlation length at imaginary $\theta$ should be real 
* Additionally, when this value is computed in C++, the results are significantly off from the calculation done in Python
* Currently, the simulation returns the average value of the correlation function computed after thermalization. There is no way to change this after the simulation is run, and we don't have the code set up yet to return any error on the correlation function. That may come in future versions of the code.

Research questions and next steps
* What can a ML model learn from the configuration data we have so far?
* How will Complex Langevin results differ from these if at all?